# 09 — Position management (open trades)
This notebook documents how open positions are evaluated and how stop updates are suggested.
It uses the same R-based logic as the rest of the system, without any auto-execution.


## Steps and outputs
1) Load positions from `positions.json`.
2) Fetch OHLCV for open tickers.
3) Evaluate positions and produce suggested actions.
4) (Optional) apply only MOVE_STOP_UP updates to your local state.

Outputs from `evaluate_positions`:
- `action`: `MOVE_STOP_UP`, `CLOSE_STOP_HIT`, `CLOSE_TIME_EXIT`, or `NO_ACTION`.
- `r_now`: current R = (last - entry) / (entry - stop).
- `stop_suggested`: only ever moves up (never down).


## Stop management rules (default `ManageConfig`)
- Breakeven: when R >= 1, move stop up to entry.
- Trail: when R >= 2, trail below SMA20 with a small buffer.
- Time exit: close after `max_holding_days` bars since entry_date.
- Stop hit: if last <= stop, mark as `CLOSE_STOP_HIT`.


In [ ]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig
from swing_screener.portfolio.state import (
    load_positions,
    evaluate_positions,
    updates_to_dataframe,
    apply_stop_updates,
    render_degiro_actions_md,
)

positions = load_positions('positions.json')
tickers = [p.ticker for p in positions if p.status == 'open']
tickers = list(dict.fromkeys([t.strip().upper() for t in tickers if t]))
if 'SPY' not in tickers:
    tickers.append('SPY')

ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start='2022-01-01'))
updates, new_positions = evaluate_positions(ohlcv, positions)
updates_df = updates_to_dataframe(updates)
updates_df


In [ ]:
# Apply only MOVE_STOP_UP suggestions to local state (does not auto-close trades)
updated_positions = apply_stop_updates(new_positions, updates)

# Generate a Degiro-friendly checklist (markdown text)
md_text = render_degiro_actions_md(updates)
md_text



## Glossary (tickers and metrics)
- SPY: ETF tracking the S&P 500. We use it as the benchmark for Relative Strength (RS).
- SMA: Simple Moving Average. Used for trend filters and pullback signals.
- ATR: Average True Range. Measures typical daily range; used for stop distance and sizing.
- RS: Relative Strength = momentum vs benchmark (e.g., stock 6m return minus SPY 6m return).
- R (R-multiple): Risk unit where 1R = entry - stop. Used in backtests and trade management.
